Modified: Jul 31, 2019

# Evolution of curves using SDF

- Define a step function that encodes the discrete PDF of the curve evolution 
- Visualize the contour lines (at zero level)
- Active Contour 
    - satellite images
    - biomedical images
- Next step: 
    - agent-based clustering for image segmentation


In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import numpy as np
import scipy as sp
from scipy.signal import correlate2d
import pandas as pd
    
from pathlib import Path
from pprint import pprint as pp
p = print

from sklearn.externals import joblib
import pdb

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr

from holoviews import opts, dim
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import Stream, param
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts

import panel as pn
import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf

hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'
pn.extension()

## Import helper functions

In [ ]:
# Add the utils directory to the search path
UTILS_DIR = Path('../utils').absolute()
assert UTILS_DIR.exists()
if str(UTILS_DIR) not in sys.path:
    sys.path.insert(0, str(UTILS_DIR))
    print(f"Added {str(UTILS_DIR)} to sys.path")

# pp(sys.path)
    

In [ ]:
from utils import get_mro as mro, nprint, timeit
import utils

import sdfs 
from samples import LSTestSample

In [ ]:
import calculus as calc
from grid import CartesianGrid

## Set visualization options

In [ ]:
H, W = 500,500

In [ ]:
opts.defaults(
    opts.Image(colorbar=True, active_tools=['wheel_zoom'], tools=['hover'],
              width=W, height=H, aspect='equal'),
    opts.Curve(tools=['hover'], active_tools=['wheel_zoom'],
              width=W, height=H, aspect='equal'),
    opts.RGB(active_tools=['wheel_zoom'], tools=['hover'],
            width=W, height=H)
)

In [ ]:
img_opts = opts.Image(height=H, width=W, colorbar_position='bottom')
vfield_opts = opts.VectorField(width=W, height=H, color='Magnitude',
#                                magnitude=dim('Magnitude').norm()*0.2,
                               pivot='tip',
                               rescale_lengths=True)
curve_opts = opts.Points(size=5,width=W, height=H, padding=0.1, 
#                             xlim=(-10,10), ylim=(-10,10),
#                         color=dim('p')*256-50
                        )
contour_opts = opts.Contours(width=W, height=H, 
                             colorbar=False, 
                             tools=['hover'])

In [ ]:
# Grab registered bokeh renderer
print("Currently available renderers: ", *hv.Store.renderers.keys())
renderer = hv.renderer('bokeh')

---
## Curve Evolution 

Todo
- [ ] Finite Difference method on parametric equations
- [ ] Levelset functions
    - 2D signed distance functions: [src](https://is.gd/t7p5mk)
- [ ] Active contour on satellite images
- [ ] Agent-based modelling with specified rules
    - satellite image segmentation (~ clustering based on local features)

### Front Propagation 
Refer to Equation 4.8 and 4.20

1. Spatial gradient computation </br>
For stability in computing the spatial gradients, use Eqn. 4.33

2. Temporal discretization </br>
To propagate the front over time, we neet to update the levelset values over time 
according to a process. We express the process as a partial differential of the levelset function $\phi$ wrt time:

$$
\frac{\partial \phi}{\partial t} (t)= L(t, \phi(t)), ~~~~~ \phi(t_0) = \phi^{0}
$$

where $L$ represents a general function that depends on time stamp $t$ and the levelset function itself.
Given $L$ and the initial condition $\phi_{0}$, our goal is to find $\phi(t_{1}), \phi(t_{2})$, ... $\phi(t_{n})$ This process is called **"time integration''**, as we are solving the partial differential equation wrt time.

Using the first-order Taylor expansion around $t + \Delta t$, we derive a "forward" Euler time integration equation for discretely sampled levelset:

$$
\begin{align}
\phi(t+\Delta t) &\approx \phi(t) + \Delta t \frac{\partial \phi}{\partial t}(t) \\
                 &\approx \phi(t) + \Delta t L(t, \phi(t))
\end{align}
$$


The equations we have described both in a continous and discretized domain for $\phi(t)$ hold for any $\phi$, and is not specificed to a levelset function.  In that respect, a better notation to express this generality, I should have used a symbol that is not $\phi$, oops. But, we are interested in the levelset functions (in particular a discretly sampled one), so from here on, we will view this general equation from the perspective of time integration for a levelset equation.  That means, $\phi$ refers to the levelset function, and $L$, which describes how the levelset fucntion changes over time, correpsonds to the speed function $F$ in the fundamental levelset form. Recall this speed function is the speed in direction normal to the levelset function (at all levels).


### Two ways to view the fundamental levelset formulation

$$
\begin{align}
\frac{\partial \phi}{\partial{t}} &= -\nabla{\phi} \cdot \vec{V}  \label{eq:4.7}  \tag{4.7} \\
                                  &= -\lVert \vec{\nabla} \phi \rVert F(\vec{x}, \vec{n}, \phi, \dots )
                                     \label{eq:4.8}  \tag{4.8} \\
\end{align}
$$

1. Advection: floating surface in an external vectorfield/flow $\vec{V}$


2. Motion of a front in normal direction according to a given speed $F$ </br>
The front propagates in its normal direction with a given speed $F(t, \phi)$

The two views are mathmatically equivalent as can be shown by using the equality that links the external vector field point of view to the levelset domain by linking the way to compute normal vector in both domains:

$$
\vec{n}(\vec{x}) = \frac{\vec{\nabla}\phi (\vec{x})}{\lVert \vec{\nabla}\phi (\vec{x}) \rVert}
$$

where $\vec{x}$ refers to a point in the embedding space. In 2Dim case (ie. for curve evolution), we can express this point with two Cartian coordinates $(x,y)$:

$$
\vec{n}(x,y) = \frac{\vec{\nabla}\phi (x,y)}{\lVert \vec{\nabla}\phi (x,y) \rVert}
$$

The key point of this equality is that it proves the two ways to view the fundamental levelset form are mathmatically equivalent. Nevertheless, their physical meanings are distinct, and we solve the equations differently when it comes to finding the computational solutions, ie. $\phi(t_{1}),\phi(t_{2}), \dots$.

The solution to the first approach (ie. advection) will be implemented in `LSEvolver.advect` method. 
The solution to the second approach (ie. front's normal motion) will be implemented in `LSEvolver. propagate` method. 

Before moving on to the implementation, let me summaruize the time integration of a discrete levelset function for each perspective. Note that superscipts are used to indicate the time steps.

1. Advection

$$
\begin{align}
             \frac{\partial \phi}{\partial{t}} &=& -\nabla{\phi} \cdot \vec{V}  \label{eq:4.7}  \tag{4.7} \\
\Rightarrow  \phi^{t + \Delta t} &=& \phi^{t} - \Delta t ( \vec{\nabla} \phi^{t} \cdot \vec{V^{t}} ) \\
\end{align}
$$

2. Front's normal motion

$$
\begin{align}
\frac{\partial \phi}{\partial{t}} &=& -\lVert \vec{\nabla} \phi \rVert F(\vec{x}, \vec{n}, \phi, \dots )
                                     \label{eq:4.8}  \tag{4.8} \\
\Rightarrow  \phi^{t+\Delta t} &=& \phi^{t} - \Delta t \lVert \vec{\nabla} \phi^{t} \rVert F^{t} \\
\end{align}
$$

### Common examples of the speed function ( $F^{t}$)
- constant: $F^{t} = 1 ~~ \forall t$,  where the domain of every $F^{t}$ is $\{ \vec{x} \in \Gamma(t) \}$

    $$
    \begin{align}
    \phi^{t+\Delta t} &=& \phi^{t} - \Delta t \lVert \vec{\nabla} \phi^{t} \rVert \cdot 1
    \end{align}
    $$
    
- smoothing motion: $F^{t} = 1-\alpha \kappa$ where $\kappa$ is the curvature of the front
    - has an effect of regularizing the front (ie. curve in 2 dim case)
    
    $$
    \begin{align}
    \phi^{t+\Delta t} &=& \phi^{t} - \Delta t \lVert \vec{\nabla} \phi^{t} \rVert \cdot (1-\alpha \kappa(\phi^{t})
    \end{align}
    $$
    
Note $F$ is independent of $t$ in both cases, which means the form of the $F$ stays the same across the entire
propagation process. That doesn't mean the exact values at $t_1$ and $t_2$ are the same though. Specifically, this implies that we need to compute $\kappa$ at each step, based on the current time's $\phi$ since $\kappa$ is a function of a levelset which is dynamically evolving over time, ie. $\phi$ has a dependency on time variable $t$.

Another comment worth mentioning here is that the first example of constant speed function $F=1$ and the second case where $F$ depends on the curvature belong to different PDE categories. The first case is known as "hyperbolic" which is a case of "Hamilton-Jacobian equation", and the second case belongs to "parabolic" equation.  The criterion to category them into these different PDE classes is the order of derivatives that $F$ is dependent on. 

- hyperbolic: $F$ depends on at most order $1$ derivative
    - eg: $F=1$, $F=\lVert \vec{\nabla} \phi^{t} \rVert$
    - the front propagation has a directionality (aka. characteristics of the PDE) </br>
    $\Rightarrow$ we need to be careful which spatial gradient to take (forward or backward)
    
    
- parabolic: $F$ depends on derivatives of order greater than $1$
    - eg: $F= \alpha \kappa$ 
    - Information related to the current spatial location ($\vec{x}$) comes from all directions </br>
    $\Rightarrow$ one way to encompass this trait is to use central difference in computing spatial gradients
    
![advection-update](../assets/update_advection_bw.jpg)
![propagation-update](../assets/update_front_propagation_bw.jpg)


We will show how these speed functions affect the front propagation behaviors in laster sections.
For now, we are ready to jump into the implementation!

---

Modified: Jul 31, 2019 
## todo:
est: ~~30mins~~ 3days?....?
- [x] check i,j -> c,r conversion
- [x] ~~incorporate Grid as a data storage (self.grid) of LevelSet class~~ make LSEvolver as a subclass of CartesianGrid
- [x] get a list of points from holoviews polydraw
- [ ] make a list of line segments from the point list
- [ ] for each line segment, make Line2d object and get polygon for the band box
- [ ] collect the band bbox points into all list, also 
- [ ] show in holoview with the linesegments and its bands

---

### Implementation

In [ ]:
################################################################################
class LSEvolver(CartesianGrid):
    """
    Levelset propagator: A levelSet evolution solved by discrete time integration
    of a PDE with a given initial levelset function, \phi(t=0). 
    """
    def __init__(self, xs, ys, data=None, t=0):
        super().__init__(xs, ys, data)
        
        self.time = t #current time
        self.delta = np.inf # average change of LS function values between consecutive time stamps
        
    @timeit        
    def run(self, F, dt, pde_class, threshold=1e-3, maxIter=1e2, collect_every=50):
        """
        Args:
        - F (2d ndarray): same shape as self.data or broadcastable to self.data's shape(eg. a constant)
        - dt (flaot): time sampling period in continuous levelset space
        - pde_class (str): 'hyperbolic' or 'parabolic'
        - threshold (float): propagation stopping criterion based on the average change in consecutive phi values 
        - maxIter (int): maximum number of iterations for the propagate steps. This takes precedence over the threshold
        - collect_every (int): interval between collecting the levelset values for visualizing the process
        """
        count = 0
        deltas = {}
        phis = {}
        while self.delta > threshold:
            if count > maxIter: 
                print("MaxIter reached: ", count)
                break
            self.propagate(F, dt, pde_class, debug=False)
            deltas[self.time] = self.delta
            count += 1
            if count%collect_every == 0:
                print(f"Running {count}th iteration")
                phis[self.time] = self.data.copy()
                
        print(f"Ran for {count} steps, for total {self.time} periods")
        print(f"\taverage delta phi: {self.delta}")
        return deltas, phis

    def propagate(self, F, dt, pde_class, debug=False):
        """
        Equation 4.8 and 4.20
        1. Spatial gradient computation
        For stability in computing the spatial gradients, use Eqn. 4.33
        
        2. Temporal discretization
        To propagate the front over time, we neet to update the levelset values over time 
        according to a process. We express the process as a partial differential of the levelset 
        function (\phi) wrt time:
        
        \frac
        
        Our goal is to find \phi(t1), \phi(t2), ... given a initial \phi(t0)
        equatation
        This process is called "time integration" 
        
        Args:
        - pde_class (str): 'hyperbolic', 'parabolic'
            * (1) if F depends on at most order 1 derivatives of the levelset function phi 
            wrt space and time, the information propagation has a specific direction 
            (ie. "characteristics"), and we need to be careful about which gradient to 
            take -- backward, forward.  In this case, the levelset equation is 'hyperbolic', 
            which is a subclass of Hamilton-Jacobian equation. 
            
            * (2) if F depends on derivatives of order >= 2 (eg. F = alpha*curvature),
            then the information propagates from all directions, and we can use the 
            central finite difference method to compute the spatial gradients.
        """
        assert pde_class in ['hyperbolic','parabolic'], f"pde_class must be either 1 or 2: {pde_class}"
        
        if pde_class == 'hyperbolic':
            dxb, dxf, dyb, dyf = self.get_diff1_bf()
            
            if debug:
                overlay = (
                    hv.Image(self.data, label='phi') + hv.Image([])
                    + hv.Image(dxb, label='dx back') + hv.Image(dxf, label='dx forward')
                    + hv.Image(dyb, label='dy back') + hv.Image(dyf, label='dy forward')
                ).cols(2)
                display(overlay)

            S = np.sign(F)
            dx = np.maximum(S*dxb, -S*dxf)
            dy = np.maximum(S*dyb, -S*dyf)
            
        else: #pde_class == 'parabolic':
            dx,dy = self.get_diff1_central()

        dmag = np.sqrt(dx**2 + dy**2)
    
        # update phi
        dphi = dt * dmag * F
        self.data -= dphi 
        self.delta = dphi.sum() / dphi.size

        # update time
        self.time += dt

        
    def advect(self, V, dt):
        """
        Args:
        - V (ndarray of shape (w,h,2)): containing x and y component of the vector field
        - dt (float): time step size
        """
        if dt > min(self.dx, self.dy):
            #todo: print error but then make dt smaller smartly
            raise ValueError('dt should be smaller than x and y sample resolutions: ', dt)
        pass
    
    def reinit(self, method='sweep'):
        """
        Reset current phi values (in self.data) to satisfy Eikonal equality
        in Eqn. 4.12
        
        - method 
            - 'pde': solve eqn. 4.37 with current phi data, until steady state
            - 'fmm': fast marching method
            - 'sweep' (default): paper [88]
            - 'exact': paper [64]
            
            Default is 'sweep'
        """
        pass
    
    def get_diff1_bf(self, switch=True):
        dxb, dxf, dyb, dyf = calc.diff1_bf(self.data, switch)
        return dxb/self.dx, dxf/self.dx, dyb/self.dy, dyf/self.dy
    
    def get_diff1_central(self):
        dx, dy= calc.diff1_central(self.data)
        return dx/(2*self.dx), dy/(2*self.dy)
    
    def get_curvature(self):
        return curvature(self.data)
    
    def satisfies_cfl(self, V, dt, method="euler1"):
        """
        todo: rename to validate_dt?
        
        Note: this is applicable only when you choose to 'advect' your front. 
        - irrelevant to 'propagate' method.
        
        Check if the given dt satisifes the CFL condition for the stability of Euler forward
        time integration (ie. explicit method). In other words, this is to ensure the time 
        steps are small enough in comparison to the spatial sampling size (dx and dy) so that 
        the errors do not grow over time.
        
         $$ \frac{V \dot \Delta t}{\Delta x} < c $$
         
         where $c$ is the CFL-number which depends on the time integration method.
         
         In case of the explicit time integration method (eg. forward Euler and RK-schemes), 
         c = 1. Other cases (ie. for higher order methods), the c value can be very restrictive, 
         and it may be better to use an implicit time integration (eg. backward Euler)
         
         Args:
         - V (3 dim np.ndarray): 
             - first channel has the x component of the external velocity field
             - second channel has the y component of the external velocity field
         - method (str): time integration method and order. 
             - currently supports only "euler1" which indicates "first-order forward Euler" 
        """
        if method is not "euler1":
            raise ValueError("Currently only first-order forward euler method is supported")
        assert V.ndim == 3, f"V should be three-dimensional: {V.ndim}"
        Vx, Vy = V[...,0], V[...,1]
        
        Vx_satisfies = np.all(Vx < dt/self.dx)
        Vy_satisfies = np.all(Vy < dt/self.dx)
        is_valid = Vx_satisfies and Vy_satisfies
        if not is_valid:
            Vmax = V.max()
            dt = (min(self.dx, self.dy) / Vmax) - 1e-5
        return (is_valid, dt)
    


## Test LSEvolver

In [ ]:
# xs,ys,zz = LSTestSample.centered_linear_grid()

n_points = 100
xlim = (-2,2)
ylim = (-2,2)
sdf = sdfs.sdUnitCircle
xs = np.linspace(*xlim, n_points)
ys = np.linspace(*ylim, n_points)[::-1]
zz = sdfs.eval_sdf(xs, ys, sdf)

In [ ]:
ls = LSEvolver(xs,ys,zz)
ls

In [ ]:
phi0_img = hv.Image((xs,ys,zz)) 
front0 = hv.Image((xs,ys,np.isclose(zz,0)))
contour0 = hv.operation.contours(phi0_img, levels=1).opts(contour_opts)
# (phi0 + front0).opts(shared_axes=False)
# (phi0_img * contour0 + front0).opts(shared_axes=False)

## Intermediate results collection containers
times = [ls.time]
phis = {ls.time: ls.data.copy()}
deltas = {ls.time: ls.delta}


In [ ]:
try: 
    del tstep_widget, mdbox
except NameError:
    pass

F = 1
dt = 1e-2
pde_class='hyperbolic'
maxIter = 500
tstep_widget = pn.widgets.Player(start=0, end=maxIter, value=0, step=1)
mdbox = pn.pane.Markdown('')

# if tstep_widgets is redefined/resassigned, any parameterized function that depends on this widget's param value 
# must be deleted and then run the below
try: 
    del step
    del show_current_phi
except NameError:
    pass


In [ ]:
@pn.depends(tstep_widget.param.value)
def step(value):
    #show_current_phi()

    
    # Propagate one step 
    out = f"""##Step: {value}, time: {ls.time:.3f} -> {ls.time+dt:.3f}
    Propagation started...
    """
    mdbox.object =  out
    ls.propagate(F,dt,pde_class)
    mdbox.object += "Propagation finished\n"
    
    # Record intermediate results
    phis[ls.time] = ls.data.copy()
    deltas[ls.time]=ls.delta
    times.append(ls.time)
    
    img = hv.Image((ls.xs,ls.ys,ls.data), group='phi', label=f'time: {ls.time}').opts(img_opts)
    contour = hv.operation.contours(img, levels=1).opts(contour_opts)
    mdbox.object += "Image updated"
    return img * contour


In [ ]:
# pn.Row(tstep_widget, mdbox, step)

In [ ]:
ls.time, ls.delta

In [ ]:
# @pn.depends(tstep_widget.param.value)
# def show_current_phi(*args, **kwargs):
#     img = hv.Image((ls.xs,ls.ys,ls.data), group='phi', label=f'time: {ls.time}').opts(img_opts)
#     contour = hv.operation.contours(img, levels=1).opts(contour_opts)
#     return img * contour
    

In [ ]:
pn.Row(
    pn.Column(tstep_widget, mdbox),
    step
)

## Simulate while recording
Another way is to simulate all at once for certain steps (either decided by the threshold value of delta or maxIter)
and visualize the collected intermediate results after the simulation finishes 

In [ ]:
n_points = 100
xlim = (-2,2)
ylim = (-2,2)
# sdf = sdfs.sdUnitCircle
sdf = sdfs.sdStar1
sdf_name = 'sdStar1'

xs = np.linspace(*xlim, n_points)
ys = np.linspace(*ylim, n_points)[::-1]
zz = sdfs.eval_sdf(xs, ys, sdf)
base = hv.Image((xs, ys, zz))

    
ls = LSEvolver(xs,ys,zz)
F = 1
dt = 1e-2
pde_class = 'hyperbolic'
collect_every = 10
maxIter = 100
threshold=1e-6
# ls.propagate(F,dt,pde_class,True)

In [ ]:
deltas, phis = ls.run(F, dt, pde_class, threshold=1e-6, maxIter=maxIter, collect_every=collect_every)

Visualize the intemediate phis and deltas 

In [ ]:
# save computation results
out_pkl = f'../data/intrim/{sdf_name}_f_{F}_dt_{dt}_t_0_{ls.time:.1f}.pkl'
joblib.dump((deltas,phis), out_pkl)

### Visualize intermediate propagation results
1. phi values

In [ ]:
hv.extension('bokeh')
tsteps= list(phis.keys())
hmap = hv.HoloMap({t: hv.Image((ls.xs, ls.ys, phis[t])) for t in tsteps})
# hmap.opts(framewise=True, fig_inches=10) # for matplotlib
hmap.opts(shared_axes=False) # for bokeh

Save animation of propagation with contours

In [ ]:
hv.extension('matplotlib')                
out = f'../outputs/levelset/{sdf_name}_f_{F}_dt_{dt}_t_0_{ls.time:.1f}' 
contour_hmap =hv.operation.contours(hmap,levels=5)
overlay = hmap * contour_hmap
overlay.opts(framewise=True, fig_inches=10)

In [ ]:
hv.save(hmap.opts(framewise=True), out + '_hmap.gif', fps=1)
hv.save(overlay.opts(framewise=True), out+'_overlay.gif', fps=1)

In [ ]:
# phis in dmap
phi_dmap = hv.DynamicMap(lambda t: hv.Image((xs,ys,phis[t])), kdims='t').redim.values(t=tsteps)
phi_dmap = phi_dmap.opts(img_opts)

In [ ]:
# dmap of contours
contour_dmap = hv.operation.contours(phi_dmap,levels=10)
contour_dmap = contour_dmap.redim.values(t=list(phis.keys())).opts(contour_opts)

2. deltas in hv.Contours

In [ ]:
# show results
try: 
    deltas.pop(0.)
except KeyError:
    print("deltas don't contain time=0 value")
    pass

curve = hv.Curve(deltas)
dmap_point = hv.DynamicMap(lambda t: hv.Points( [(t, deltas[t])] ),
                           kdims='t')
dmap_point = dmap_point.redim.values(t=list(deltas.keys()))

(curve * dmap_point.opts(color='red', size=5)).redim.label(x='time', y='delta')

In [ ]:
# Interactive simulation
hv.extension('bokeh')
selected_curve = (curve * dmap_point).redim.values(t=tsteps).redim.label(x='time', y='delta')
overlay = ( phi_dmap * contour_dmap + selected_curve )
overlay.opts(img_opts, contour_opts).opts(opts.Points(color='red', size=5))

# todo:
- [x] use Run method 
- [ ] debug why it's moving up

In [ ]:
def test_levelset_unit_circle(to_save=False):
    n_points = 100
    xlim = (-2,2)
    ylim = (-2,2)
    sdf = sdfs.sdUnitCircle
    sef_name = 'sdUnitCircle'
    
    xs = np.linspace(*xlim, n_points)
    ys = np.linspace(*ylim, n_points)[::-1]
    zz = sdfs.eval_sdf(xs, ys, sdf)
    ls = LSEvolver(xs,ys,zz)
    
    # propagation config
    F = 1
    dt = 1e-3
    maxIter = 300
    collect_every = 10
    threshold=1e-6
    
    deltas, phis = ls.run(F, dt, pde_class, threshold=1e-6, maxIter=maxIter, collect_every=collect_every)
    
    # save result
    if to_save:
        out_pkl = f'../data/intrim/{sdf_name}_f_{F}_dt_{dt}_t_0_{ls.time:.1f}.pkl'
        
    return deltas, phis


In [ ]:
def test_levelset_star1(to_save=False):
    n_points = 100
    xlim = (-2,2)
    ylim = (-2,2)
    sdf = sdfs.sdStar1
    sdf_name = 'sdStar1'
    
    xs = np.linspace(*xlim, n_points)
    ys = np.linspace(*ylim, n_points)[::-1]
    zz = sdfs.eval_sdf(xs, ys, sdf)
    ls = LSEvolver(xs,ys,zz)
    
    # propagation config
    F = 1
    dt = 1e-3
    maxIter = 300
    collect_every = 10
    threshold=1e-6
    
    deltas, phis = ls.run(F, dt, pde_class, threshold=threshold, maxIter=maxIter, collect_every=collect_every)
    
    # save result
    if to_save:
        out_pkl = f'../data/intrim/{sdf_name}_f_{F}_dt_{dt}_t_0_{ls.time:.1f}.pkl'   
        
    return deltas, phis

def test_levelset_star2(to_save=False):
    n_points = 100
    xlim = (-2,2)
    ylim = (-2,2)
    
    sdf = sdfs.sdStar2
    sdf_name = 'sdStar2'
    
    xs = np.linspace(*xlim, n_points)
    ys = np.linspace(*ylim, n_points)[::-1]
    zz = sdfs.eval_sdf(xs, ys, sdf)
    ls = LSEvolver(xs,ys,zz)
    
    # propagation config
    F = 1
    dt = 1e-3
    maxIter = 300
    collect_every = 10
    threshold=1e-6

    deltas, phis = ls.run(F, dt, pde_class, threshold=threshold, maxIter=maxIter, collect_every=collect_every)
    
    # save result
    if to_save:
        out_pkl = f'../data/intrim/{sdf_name}_f_{F}_dt_{dt}_t_0_{ls.time:.1f}.pkl'
    
    return deltas, phis

In [ ]:
to_save = True

unit_circle_result = test_levelset_unit_circle(to_save)
unit_star1_result = test_levelset_star1(to_save)
unit_star2_result = test_levelset_star2(to_save)

In [ ]:
def save_animation(deltas, phis):
    